# Analysis 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from scipy import stats

pd.set_option("display.precision", 3)

## Results - MAPLE can generalize with only X of 152 APLs

### Data Loading

In [2]:
path_results = '../activation-prediction/results/tcr_specific_data_size.csv.gz'
ppdf = pd.read_csv(path_results, compression='gzip')

def compute_metrics(g):
    return pd.Series({
        'MAE': g['abserr'].mean(),
        'R2': metrics.r2_score(g['act'], g['pred']),
        'Pearson': g['act'].corr(g['pred'], method='pearson'),
        'Spearman': g['act'].corr(g['pred'], method='spearman'),
        'AUC': (
            metrics.roc_auc_score(g['is_activated'], g['pred_prob'])
            if np.isfinite(g['pred_prob']).all() and 0 < g['is_activated'].mean() < 1
            else np.nan
        ),
        'APS': (
            metrics.average_precision_score(g['is_activated'], g['pred_prob'])
            if np.isfinite(g['pred_prob']).all()
            else np.nan
        ),
    })


# compute metric for each validation fold separately
mdf = pd.concat([
    # except for lmo CV where each validation fold contained a single sample
    # in that case we just compute a global average for each tcr
    ppdf.query('features=="lmo"') \
        .groupby(['features', 'normalization', 'tcr']) \
        .apply(compute_metrics).reset_index(),
    ppdf.query('features!="lmo"') \
        .groupby(['features', 'normalization', 'tcr', 'fold']) \
        .apply(compute_metrics).reset_index(),
])

mdf

C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:8

C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:8

C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:8

,features,normalization,tcr,MAE,R2,Pearson,Spearman,AUC,APS,fold
0,lmo,AS,B11,8.874,0.792,0.890,0.888,0.958,0.959,NaN
1,lmo,AS,B13,19.512,0.246,0.506,0.733,0.890,0.575,NaN
2,lmo,AS,B15,11.233,0.655,0.809,0.788,0.905,0.926,NaN
3,lmo,AS,B3,8.604,0.434,0.663,0.640,0.900,0.580,NaN
4,lmo,AS,E8,14.075,0.332,0.579,0.701,0.842,0.528,NaN
...,...,...,...,...,...,...,...,...,...,...
2371,lpo,AS,OTI_PH,19.214,-1.742,-0.097,-0.170,0.592,0.322,3.0
2372,lpo,AS,OTI_PH,20.171,-8.940,0.388,0.433,NaN,1.000,4.0
2373,lpo,AS,OTI_PH,18.143,-0.273,0.023,0.201,0.575,0.310,5.0
2374,lpo,AS,OTI_PH,14.414,-0.127,-0.112,-0.144,0.466,0.425,6.0


In [3]:
mdf_educated = mdf[mdf['tcr'].str.startswith('ED')]
mdf_naive = mdf[~mdf['tcr'].str.startswith('ED')]
mdf_naive = mdf_naive[~mdf_naive['tcr'].isin(['OTI_PH', 'LR_OTI_2', 'LR_OTI_1'])]

In [4]:
mdf.head(20)

,features,normalization,tcr,MAE,R2,Pearson,Spearman,AUC,APS,fold
0,lmo,AS,B11,8.874,0.792,0.890,0.888,0.958,0.959,NaN
1,lmo,AS,B13,19.512,0.246,0.506,0.733,0.890,0.575,NaN
2,lmo,AS,B15,11.233,0.655,0.809,0.788,0.905,0.926,NaN
3,lmo,AS,B3,8.604,0.434,0.663,0.640,0.900,0.580,NaN
4,lmo,AS,E8,14.075,0.332,0.579,0.701,0.842,0.528,NaN
5,lmo,AS,ED10,11.474,0.642,0.802,0.772,0.918,0.949,NaN
6,lmo,AS,ED16_1,10.641,0.790,0.889,0.836,0.957,0.972,NaN
7,lmo,AS,ED16_30,11.930,0.732,0.856,0.834,0.911,0.938,NaN
8,lmo,AS,ED21,10.305,0.705,0.841,0.750,0.959,0.990,NaN
9,lmo,AS,ED23,19.187,0.481,0.696,0.696,0.896,0.955,NaN


### Get Values

In [34]:
def get_metric(split, metric, repertoire='educated'):
    mdf_tmp = mdf_naive
    if repertoire == 'educated':
        mdf_tmp = mdf_educated
    elif repertoire == 'full':
        mdf_tmp = mdf
    split_perf = mdf_tmp[mdf_tmp['features']==split][metric].values
    split_perf = split_perf[~np.isnan(split_perf)]
    split_avg = np.mean(split_perf)
    
    lmo_perf = mdf_tmp[mdf_tmp['features']=='lmo'][metric].values
    lmo_perf = lmo_perf[~np.isnan(lmo_perf)]
    lmo_avg = np.mean(lmo_perf)
    
    md = split_perf.mean() - lmo_perf.mean()
    sd = lmo_perf.std()
    sd = np.sqrt((split_perf.var() + lmo_perf.var()) / 2)
    effect_size = md / sd
    
    return split_avg, lmo_avg-split_avg, effect_size

In [6]:
mdf['features'].unique()

array(['lmo', 'l10o', 'l25o', 'l50o', 'l75o', 'l90o', 'l95o', 'lao',
       'lpo'], dtype=object)

In [7]:
def get_evaluation_summary(metric, repertoire):
    res = {
        'split': [],
        'avg': [],
        'decrease': [],
        'effect size': [],
    }
    for split in mdf['features'].unique():
        avg, dec, es = get_metric(split, metric, repertoire)
        res['split'].append(split)
        res['avg'].append(avg)
        res['decrease'].append(dec)
        res['effect size'].append(es)
    return pd.DataFrame(res)  

In [8]:
get_evaluation_summary('AUC', 'educated')

,split,avg,decrease,effect size
0,lmo,0.902,0.000,0.000
1,l10o,0.905,-0.004,0.046
2,l25o,0.911,-0.009,0.193
3,l50o,0.899,0.003,-0.058
4,l75o,0.880,0.022,-0.420
5,l90o,0.826,0.076,-1.158
6,l95o,0.779,0.123,-1.549
7,lao,0.923,-0.022,0.244
8,lpo,0.519,0.383,-2.400


In [9]:
get_evaluation_summary('Spearman', 'educated')

,split,avg,decrease,effect size
0,lmo,0.797,0.000,0.000
1,l10o,0.772,0.025,-0.308
2,l25o,0.795,0.002,-0.032
3,l50o,0.780,0.017,-0.344
4,l75o,0.747,0.050,-1.002
5,l90o,0.645,0.152,-1.926
6,l95o,0.540,0.257,-1.832
7,lao,0.793,0.004,-0.035
8,lpo,0.104,0.693,-3.709


In [10]:
get_evaluation_summary('AUC', 'naive')

,split,avg,decrease,effect size
0,lmo,0.824,0.000e+00,0.000
1,l10o,0.863,-3.812e-02,0.213
2,l25o,0.841,-1.658e-02,0.092
3,l50o,0.825,-2.418e-04,0.001
4,l75o,0.770,5.433e-02,-0.316
5,l90o,0.760,6.481e-02,-0.409
6,l95o,0.706,1.185e-01,-0.758
7,lao,0.890,-6.512e-02,0.365
8,lpo,0.563,2.611e-01,-1.401


In [11]:
get_evaluation_summary('Spearman', 'naive')

,split,avg,decrease,effect size
0,lmo,0.643,0.000,0.000
1,l10o,0.639,0.004,-0.020
2,l25o,0.642,0.001,-0.005
3,l50o,0.626,0.017,-0.088
4,l75o,0.558,0.085,-0.401
5,l90o,0.461,0.182,-0.880
6,l95o,0.352,0.291,-1.322
7,lao,0.634,0.009,-0.036
8,lpo,0.089,0.554,-2.264


In [12]:
get_evaluation_summary('AUC', 'full')

,split,avg,decrease,effect size
0,lmo,0.882,0.000e+00,0.000
1,l10o,0.898,-1.696e-02,0.144
2,l25o,0.892,-1.021e-02,0.094
3,l50o,0.881,7.226e-04,-0.007
4,l75o,0.853,2.828e-02,-0.260
5,l90o,0.818,6.335e-02,-0.603
6,l95o,0.775,1.066e-01,-0.954
7,lao,0.915,-3.363e-02,0.275
8,lpo,0.542,3.394e-01,-1.955


In [35]:
get_evaluation_summary('Spearman', 'full')

,split,avg,decrease,effect size
0,lmo,0.74821,0.00000,0.00000
1,l10o,0.73456,0.01365,-0.08925
2,l25o,0.74617,0.00204,-0.01433
3,l50o,0.73208,0.01612,-0.11677
4,l75o,0.68714,0.06107,-0.40024
5,l90o,0.59079,0.15741,-0.99305
6,l95o,0.49227,0.25593,-1.35107
7,lao,0.74196,0.00625,-0.03304
8,lpo,0.09082,0.65738,-3.09418


In [14]:
mdf[mdf['features']=='lpo']

,features,normalization,tcr,MAE,R2,Pearson,Spearman,AUC,APS,fold
2160,lpo,AS,B11,30.226,-10.231,-0.050,-0.058,NaN,NaN,0.0
2161,lpo,AS,B11,14.698,-1.363,0.415,0.252,NaN,1.000,1.0
2162,lpo,AS,B11,24.545,-0.950,0.125,0.119,0.522,0.521,2.0
2163,lpo,AS,B11,36.328,-17.051,0.297,0.312,NaN,NaN,3.0
2164,lpo,AS,B11,25.459,-5.243,0.281,0.319,NaN,1.000,4.0
...,...,...,...,...,...,...,...,...,...,...
2371,lpo,AS,OTI_PH,19.214,-1.742,-0.097,-0.170,0.592,0.322,3.0
2372,lpo,AS,OTI_PH,20.171,-8.940,0.388,0.433,NaN,1.000,4.0
2373,lpo,AS,OTI_PH,18.143,-0.273,0.023,0.201,0.575,0.310,5.0
2374,lpo,AS,OTI_PH,14.414,-0.127,-0.112,-0.144,0.466,0.425,6.0


## Results acitve learning

In [15]:
import json

In [16]:
def read_act_results(method, tag):
    with open(f'../activation-prediction/results/al/{tag}_{method}_8.json') as f:
        results = json.load(f)
    res = {}
    for metric, items in results.items():
        res[metric] = {}
        for item in items:
            if item[2] not in res[metric]:
                res[metric][item[2]] = []
            res[metric][item[2]].append(item[3])
        for iteration, items in res[metric].items():
            res[metric][iteration] = sum(items)/len(items)
    samples = {0: 9}
    samples.update({i: 9+i*8 for i in range(1, 10)})
    res['Samples'] = samples
    res = pd.DataFrame(res)
    return res

In [17]:
read_act_results('act_avg', 'TEST')

,MAE,R2,Pearson,Spearman,auc,aps,F1,Accuracy,Precision,Recall,Samples
0,20.669,0.162,0.627,0.637,0.740,0.728,0.438,0.686,0.784,0.407,9
1,18.103,0.311,0.657,0.645,0.762,0.740,0.539,0.715,0.733,0.513,17
2,16.853,0.396,0.702,0.681,0.800,0.764,0.597,0.733,0.724,0.561,25
3,15.384,0.489,0.740,0.717,0.827,0.791,0.674,0.777,0.785,0.640,33
4,14.694,0.534,0.760,0.734,0.843,0.807,0.714,0.800,0.793,0.679,41
5,14.098,0.569,0.776,0.747,0.857,0.817,0.743,0.817,0.834,0.706,49
6,13.676,0.596,0.792,0.758,0.868,0.833,0.765,0.829,0.835,0.733,57
7,13.204,0.619,0.804,0.765,0.880,0.844,0.790,0.841,0.845,0.762,65
8,12.881,0.637,0.812,0.768,0.889,0.853,0.804,0.850,0.847,0.782,73
9,12.483,0.659,0.823,0.769,0.898,0.863,0.826,0.865,0.857,0.813,81


In [18]:
read_act_results('rdm', 'TEST')

,MAE,R2,Pearson,Spearman,auc,aps,F1,Accuracy,Precision,Recall,Samples
0,20.145,0.083,0.517,0.524,NaN,NaN,0.552,0.666,0.624,0.611,9
1,17.129,0.316,0.652,0.639,0.803,0.760,0.632,0.734,0.703,0.650,17
2,15.557,0.433,0.709,0.690,0.823,0.780,0.667,0.765,0.753,0.663,25
3,14.675,0.488,0.734,0.713,0.837,0.787,0.679,0.773,0.736,0.680,33
4,14.105,0.525,0.751,0.727,0.845,0.798,0.704,0.784,0.763,0.696,41
5,13.648,0.552,0.762,0.736,0.854,0.811,0.721,0.792,0.782,0.707,49
6,13.282,0.574,0.772,0.747,0.860,0.818,0.727,0.797,0.777,0.714,57
7,13.049,0.585,0.775,0.753,0.864,0.822,0.740,0.808,0.790,0.719,65
8,12.765,0.601,0.784,0.759,0.865,0.826,0.737,0.807,0.787,0.716,73
9,12.623,0.609,0.788,0.759,0.872,0.833,0.757,0.815,0.800,0.738,81


## P values

In [19]:
import scipy.stats as stats

In [70]:
def get_pvalues(split, metric, repertoire='educated'):
    mdf_tmp = mdf_naive
    if repertoire == 'educated':
        mdf_tmp = mdf_educated
    elif repertoire == 'full':
        mdf_tmp = mdf
    
    lmo_perf = mdf_tmp[mdf_tmp['features']=='lmo'][metric].values
    lmo_perf = lmo_perf[~np.isnan(lmo_perf)]
    
    split_perf = mdf_tmp[mdf_tmp['features']==split]
    #result = stats.ttest_ind(split_perf[metric].values, lmo_perf, equal_var=False)
    result = stats.ttest_1samp(split_perf[metric].values, 0.)
    print(result)
get_pvalues('lpo', 'Spearman', 'educated')

Ttest_1sampResult(statistic=4.353341197849668, pvalue=2.8536375680611897e-05)


# Results - MAPLE learns biologically relevant interactions

Note: Positions in this notebook are 0-indexed, while in the paper they are 1-indexed. E.g. pos_3 here corresponds to position 4 in the paper.

In [21]:
path_lpo = '../activation-prediction/results/tcr_stratified_leave_position_out_performance.csv.gz'
res_lpo = pd.read_csv(path_lpo, compression='gzip')
res_lpo

,tcr,mut_pos,mut_ami,activation,is_activated,pred
0,ED5,0,A,25.660,False,0.268
1,ED5,0,C,12.800,False,0.264
2,ED5,0,D,50.123,True,0.240
3,ED5,0,E,78.235,True,0.252
4,ED5,0,F,5.509,False,0.248
...,...,...,...,...,...,...
5771,B6,7,S,0.333,False,0.096
5772,B6,7,T,0.346,False,0.084
5773,B6,7,V,0.253,False,0.088
5774,B6,7,W,0.377,False,0.100


In [22]:
len(res_lpo['tcr'].unique())

38

In [23]:
path_perm = '../activation-prediction/results/tcr_stratified_permutation_importance.csv.gz'
res_perm = pd.read_csv(path_perm, compression='gzip')

In [24]:
pd.set_option("precision", 5)

In [25]:
def get_feature_importance_perm():
    path_perm = '../activation-prediction/results/tcr_stratified_permutation_importance.csv.gz'
    pdf = pd.read_csv(path_perm, compression='gzip')
    mdf = pdf[pdf['tcr'].isin(
        pdf[pdf['is_activated'] > 0.5]['tcr'].unique()
    )].groupby(['tcr', 'group', 'shuffle']).apply(lambda q: pd.Series({
        'auc': metrics.roc_auc_score(q['is_activated'], q['pred']),
        #'aps': metrics.average_precision_score(q['is_activated'], q['pred']),
        #'spearman': stats.spearmanr(q['activation'], q['pred'])[0],
    })).reset_index().drop(columns='shuffle')


    #%%
    ddf = mdf.melt(['tcr', 'group']).merge(
        mdf[
            mdf['group'] == 'all'
        ].drop(columns='group').melt('tcr', value_name='base').drop_duplicates(),
        on=['tcr', 'variable']
    )
    ddf['diff'] = ddf['value'] - ddf['base']
    ddf['rel'] = ddf['value'] / ddf['base'] - 1  # positive = increase
    ddf['item'] = ddf['group'].str.split('_').str[0]
    ddf['is_educated'] = np.where(
        ddf['tcr'].str.startswith('ED'),
        'Educated', 'Naive'
    )
    ddf = ddf.groupby(['group', 'is_educated']).mean(['value'])
    return ddf

In [26]:
res_perm = get_feature_importance_perm()
res_perm

value     base     diff      rel
group is_educated                                    
all   Educated     0.90307  0.90307  0.00000  0.00000
      Naive        0.64381  0.64381  0.00000  0.00000
cdr3  Educated     0.65647  0.90307 -0.24660 -0.27202
      Naive        0.55715  0.64381 -0.08667  0.12049
cdr3a Educated     0.86924  0.90307 -0.03383 -0.03742
      Naive        0.63445  0.64381 -0.00936  0.01429
cdr3b Educated     0.81951  0.90307 -0.08356 -0.09226
      Naive        0.61159  0.64381 -0.03223  0.04442
pos_0 Educated     0.87447  0.90307 -0.02859 -0.03160
      Naive        0.62284  0.64381 -0.02098 -0.02545
pos_1 Educated     0.87339  0.90307 -0.02967 -0.03375
      Naive        0.59977  0.64381 -0.04404 -0.06976
pos_2 Educated     0.88603  0.90307 -0.01704 -0.01850
      Naive        0.63160  0.64381 -0.01221 -0.02077
pos_3 Educated     0.77880  0.90307 -0.12426 -0.13671
      Naive        0.62056  0.64381 -0.02325  0.22709
pos_4 Educated     0.88702  0.90307 -0.01605 -0.01817
      Naive        0.63503  0.64381 -0.00878  0.01784
pos_5 Educated     0.81461  0.90307 -0.08845 -0.09721
      Naive        0.61275  0.64381 -0.03106 -0.06248
pos_6 Educated     0.87371  0.90307 -0.02936 -0.03173
      Naive        0.65601  0.64381  0.01219  0.10344
pos_7 Educated     0.88755  0.90307 -0.01551 -0.01724
      Naive        0.64192  0.64381 -0.00190  0.02461

In [27]:
def get_feature_importance_lpo():
    path_in = '../activation-prediction/results/tcr_stratified_leave_position_out_performance.csv.gz'
    pdf = pd.read_csv(path_in, compression='gzip')
    pp = pdf.groupby(
        ['tcr', 'mut_pos'], as_index=False
    ).apply(lambda q: pd.Series({
        'auc': metrics.roc_auc_score(
            q['is_activated'], q['pred']
        ) if 0 < q['is_activated'].mean() < 1 else np.nan,
        'aps': metrics.average_precision_score(q['is_activated'], q['pred']),
    }))

    pp['is_educated'] = np.where(
        pp['tcr'].str.startswith('ED'),
        'Educated', 'Naive'
    )
    pp = pp.groupby(['mut_pos', 'is_educated']).mean(['auc', 'aps'])
    return pp

In [28]:
res_lpo = get_feature_importance_lpo()

C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:8

C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\felix.drost\Anaconda3\envs\mutagenesis\lib\site-packages\sklearn\metrics\_ranking.py:8

In [29]:
res_lpo

auc      aps
mut_pos is_educated                  
0       Educated     0.50574  0.47896
        Naive        0.47428  0.23789
1       Educated     0.65295  0.96894
        Naive        0.57214  0.83144
2       Educated     0.56024  0.65749
        Naive        0.53796  0.62058
3       Educated     0.42623  0.22552
        Naive        0.40837  0.16208
4       Educated     0.62549  0.91673
        Naive        0.69692  0.87409
5       Educated     0.39948  0.29937
        Naive        0.51600  0.29956
6       Educated     0.56089  0.50464
        Naive        0.54710  0.29986
7       Educated     0.49248  0.89158
        Naive        0.42371  0.71766

In [30]:
res_lpo['auc'] - res_perm['base'] 

is_educated  mut_pos  group
Educated     0        all     -0.39733
                      cdr3    -0.39733
                      cdr3a   -0.39733
                      cdr3b   -0.39733
                      pos_0   -0.39733
                                ...   
Naive        7        pos_3   -0.22010
                      pos_4   -0.22010
                      pos_5   -0.22010
                      pos_6   -0.22010
                      pos_7   -0.22010
Length: 192, dtype: float64

In [31]:
res_perm_tmp = res_perm.unstack()
res_perm_tmp = res_perm_tmp[res_perm_tmp.index.str.startswith('pos_')]
res_perm_tmp.index = list(range(8))
res_perm_tmp.index.name = 'mut_pos'
res_perm_tmp = res_perm_tmp.stack()
res_lpo['diff'] = res_perm_tmp['base'].values - res_lpo['auc'].values
res_lpo

auc      aps     diff
mut_pos is_educated                           
0       Educated     0.50574  0.47896  0.39733
        Naive        0.47428  0.23789  0.16954
1       Educated     0.65295  0.96894  0.25011
        Naive        0.57214  0.83144  0.07167
2       Educated     0.56024  0.65749  0.34283
        Naive        0.53796  0.62058  0.10585
3       Educated     0.42623  0.22552  0.47684
        Naive        0.40837  0.16208  0.23545
4       Educated     0.62549  0.91673  0.27757
        Naive        0.69692  0.87409 -0.05311
5       Educated     0.39948  0.29937  0.50359
        Naive        0.51600  0.29956  0.12781
6       Educated     0.56089  0.50464  0.34218
        Naive        0.54710  0.29986  0.09671
7       Educated     0.49248  0.89158  0.41058
        Naive        0.42371  0.71766  0.22010